# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import csv
import openai
from openai import OpenAI
from tqdm import tqdm
import time

In [ ]:
# Your OpenAI API key
client = OpenAI(
  base_url="****",
  api_key="****",
)

# Reading File

In [ ]:
# Reading file
model_name = pd.read_csv("zero-shot-generated-{model_name}.csv")  

# Needing sentence column
sentences = model_name['Sentence']
sentences.head()

# LLMs as judge

In [ ]:
# LLM as judge
MODEL = 'anthropic/claude-3.7-sonnet'

# Score Originality 

In [ ]:
def claude_score_originality(sentence):
    prompt = f"""
    شما باید خلاقیت یک جمله را بر اساس ۳ زیرمعیار مرتبط با "نوآوری" ارزیابی کنید.  
    لطفاً برای هر معیار یک عدد بین **۱ تا ۵** بدهید. **فقط عدد را بدهید و هیچ توضیحی اضافه نکنید.
    کمترین نمره عدد ۱ و بیشترین نمره عدد ۵ است**  

    1. **آیا جمله خلاقانه است و از کلیشه های رایج فاصله دارد؟**  
    - ۱: جمله کاملاً کلیشه‌ای و بدون نوآوری است.  
    - ۲: جمله کمی از کلیشه فاصله دارد اما همچنان تکراری به نظر می‌رسد.  
    - ۳: جمله نسبتاً خلاقانه است اما هنوز مشابه جملات رایج است.  
    - ۴: جمله نوآورانه است اما شباهت‌هایی جزئی دارد.  
    - ۵: جمله کاملاً نوآورانه و غیرکلیشه‌ای است.  

    2. **آیا این جمله شبیه به جملات مشهور، اشعار یا ضر بالمث لهای فارسی است؟**  
    - ۱: جمله کاملاً مشابه یک ضرب‌المثل، شعر یا جمله‌ی معروف است.  
    - ۲: جمله شباهت زیادی به متون معروف دارد اما اندکی تغییر کرده است.  
    - ۳: جمله ساختار متفاوتی دارد اما هنوز تحت تأثیر متون معروف است.  
    - ۴: جمله تأثیرپذیری کمی از متون معروف دارد اما کاملاً مستقل نیست.  
    - ۵: جمله کاملاً منحصربه‌فرد و بدون شباهت به متون معروف است.  

    3. **آیا جمله دارای حداقل یکی از آرایه های ادبی تشبیه، استعاره، تضاد یا اغراق است؟**  
    - ۱: جمله کاملاً ساده و بدون هیچ آرایه‌ی ادبی است.  
    - ۲: جمله دارای یک آرایه‌ی ادبی است.  
    - ۳: جمله از دو آرایه‌ی ادبی استفاده کرده است.  
    - ۴: جمله از سه آرایه‌ی ادبی استفاده کرده است.  
    - ۵: جمله از چهار آرایه‌ی ادبی استفاده کرده است.  
    
    **مهم:** لطفاً برای هر سوال یک امتیاز عددی بین 1 تا 5 بدهید و فقط عدد بنویسید (مثلاً 1، 2، 3، 4 یا 5). هیچ توضیحی ندهید و از اعشاری (مانند 4.5 یا 1.33) استفاده نکنید.

    **مهم:** فقط در این فرمت پاسخ دهید و هیچ توضیحی اضافه نکنید:  
    کلیشه‌ای بودن: X  
    شباهت به متون معروف: X  
    آرایه‌های ادبی: X  

    جمله: "{sentence}"  
    """  

    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "شما یک کارشناس در ارزیابی خلاقیت متون ادبی هستید."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
        )

        generated_text = response.choices[0].message.content.strip().lower()
        print(generated_text)

        # استخراج نمرات از پاسخ
        scores = {}
        for line in generated_text.split("\n"):
            if ":" in line:
                key, value = line.split(":")
                key, value = key.strip(), value.strip()

                if key in ["کلیشه‌ای بودن", "شباهت به متون معروف", "آرایه‌های ادبی"]:
                    try:
                        value = int(value)
                        if 1 <= value <= 5:
                            scores[key] = value
                        else:
                            scores[key] = None
                    except ValueError:
                        scores[key] = None

        # Check if all metrics have valid score
        if len(scores) != 3:
            return {crit: None for crit in ["کلیشه‌ای بودن", "شباهت به متون معروف", "آرایه‌های ادبی"]}

        return scores

    except Exception as e:
        print(f"❌ خطا در پردازش جمله: {sentence}\n🔹 خطا: {str(e)}")
        return {crit: None for crit in ["کلیشه‌ای بودن", "شباهت به متون معروف", "آرایه‌های ادبی"]}


In [ ]:
# Scores sentences by Claude 3.7 sonnet
scores_list_originality = []
criteria = ["کلیشه‌ای بودن", "شباهت به متون معروف", "آرایه‌های ادبی"]  # همانطور که در تابع get_gpt_score تعریف کردیم
for sentence in tqdm(df["Sentence"], desc="Scoring Sentences with Claude 3.7 Sonnet"):
    print(sentence)
    scores = claude_score_originality(sentence)
    scores["sentence"] = sentence
    
    if all(v is not None for v in scores.values()):  # If all metrics have valid score
        # Total score as Creativity score
        scores["Originality Avg"] = sum(scores[crit] for crit in criteria) / len(criteria)
    else:
        scores["Originality Avg"] = None  # اگر نمره‌ای معتبر نبود

    scores_list_originality.append(scores)
    time.sleep(1)  # API Rate Limit

# Convert to DataFrame for analyzing
org_df = pd.DataFrame(scores_list_originality)


# Score Fluency

In [ ]:
# Get scores for fluency metric by claude
def claude_score_fluency(sentence):  
    prompt = f"""  
    شما باید خلاقیت یک جمله را بر اساس ۳ زیرمعیار مرتبط با "نوآوری" ارزیابی کنید.  
    لطفاً برای هر معیار یک عدد بین **۱ تا ۵** بدهید. **فقط عدد را بدهید و هیچ توضیحی اضافه نکنید.
    کمترین نمره عدد ۱ و بیشترین نمره عدد ۵ است** 

    1. آیا جمله از نظر دستوری صحیح و قابل فهم است؟  
       - جمله از نظر دستوری کاملاً اشتباه است و فهم آن دشوار است.  
       - جمله دارای چندین اشتباه دستوری است که درک آن را سخت می‌کند.  
       - جمله تقریباً درست است اما اشتباهات کوچکی دارد.  
       - جمله از نظر دستوری صحیح است اما می‌توان آن را کمی بهبود داد.  
       - جمله کاملاً درست و واضح است.  

    2. آیا جمله برای خواننده فارسی‌زبان روان و طبیعی به نظر می‌رسد؟  
       - جمله بسیار غیرطبیعی و مصنوعی است.  
       - جمله به سختی قابل درک است و از نظر زبانی کمی خشک است.  
       - جمله مفهوم خود را منتقل می‌کند اما هنوز به اندازه کافی روان نیست.  
       - جمله روان است، فاقد اشکال دستوری است و به راحتی خوانده می‌شود، اما ممکن است از جذابیت یا عمق بیشتری برخوردار شود.  
       - جمله کاملاً روان، طبیعی و جذاب است و هیچ بهبودی نیاز ندارد.  

    3. آیا می‌توان جمله را در یک متن ادبی یا مکالمه روزمره به کار برد؟  
       - جمله نامفهوم است و هیچ کاربردی ندارد.  
       - جمله تا حدی قابل استفاده است اما غیرطبیعی به نظر می‌رسد.  
       - جمله در متون خاصی قابل استفاده است اما برای مکالمه روزمره نامناسب است.  
       - جمله در مکالمات قابل استفاده است اما در متون ادبی قابل نامناسب است.  
       - جمله به طور کامل هم در ادبیات و هم در مکالمات کاربرد دارد.

    **مهم:** لطفاً برای هر سوال یک امتیاز عددی بین 1 تا 5 بدهید و فقط عدد بنویسید (مثلاً 1، 2، 3، 4 یا 5). هیچ توضیحی ندهید و از اعشاری (مانند 4.5 یا 1.33) استفاده نکنید.

    جمله: "{sentence}"  

    لطفاً پاسخ خود را در فرمت زیر بدهید:  
    دستوری صحیح: X  
    روانی و طبیعی بودن: X  
    کاربرد در مکالمه و ادبیات: X
    """  

    try:  
        response = client.chat.completions.create(  
            model=MODEL,  
            messages=[  
                {"role": "system", "content": "شما یک کارشناس در ارزیابی خلاقیت متون ادبی هستید."},  
                {"role": "user", "content": prompt}  
            ],  
            max_tokens=50,  
        )  
        
        generated_text = response.choices[0].message.content.strip().lower()  
        print(generated_text)  
        
        # Extract scores from response 
        scores = {}  
        for line in generated_text.split("\n"):  
            if ":" in line:  
                key, value = line.split(":")  
                key, value = key.strip(), value.strip()  

                # Checking key  
                if key in ["دستوری صحیح", "روانی و طبیعی بودن", "کاربرد در مکالمه و ادبیات"]:  
                    try:  
                        value = int(value)  # Just int 
                        if 1 <= value <= 5:  
                            scores[key] = value  
                        else:  
                            scores[key] = None   
                    except ValueError:  
                        scores[key] = None  

        if len(scores) != 3:  
            return {crit: None for crit in ["دستوری صحیح", "روانی و طبیعی بودن", "کاربرد در مکالمه و ادبیات"]}  

        return scores  

    except Exception as e:  
        print(f"❌ خطا در پردازش جمله: {sentence}\n🔹 خطا: {str(e)}")  
        return {crit: None for crit in ["دستوری صحیح", "روانی و طبیعی بودن", "کاربرد در مکالمه و ادبیات"]}


In [ ]:
scores_list_fluency = []
fluency_criteria = ["دستوری صحیح", "روانی و طبیعی بودن", "کاربرد در مکالمه و ادبیات"]  
for sentence in tqdm(df["Sentence"], desc="Scoring Sentences with Claude3.7-sonnet"):
    fluency_scores = claude_score_fluency(sentence)
    fluency_scores["sentence"] = sentence
    
    if all(v is not None for v in fluency_scores.values()):  
        fluency_scores["Fluency Score"] = sum(fluency_scores[crit] for crit in fluency_criteria) / len(fluency_criteria)
    else:
        fluency_scores["Fluency Score"] = None 

    scores_list_fluency.append(fluency_scores)
    time.sleep(1)  

fluency_df = pd.DataFrame(scores_list_fluency)


In [ ]:
merge_fluency = pd.merge(org_df, fluency_df, on='sentence',how='left')

# Score Flexibility

In [ ]:
def claude_score_flexibility(sentence):  
    prompt = f"""  
    شما باید انعطاف‌پذیری یک جمله را بر اساس ۳ زیرمعیار مرتبط با "انعطاف‌پذیری" ارزیابی کنید.  
    لطفاً برای هر معیار یک عدد بین **۱ تا ۵** بدهید. **فقط عدد را بدهید و هیچ توضیحی اضافه نکنید.**
    کمترین نمره عدد ۱ و بیشترین نمره عدد ۵ است** 

    1. آیا جمله از چندین ایده برای بیان مو ضوع مورد نظر استفاده کرده است ؟
       • جمله از هیچ ایده ای برای بیان موضوع استفاده نکرده است.
       • جمله از یک ایده استفاده کرده و به خوبی نتوانسته با موضوع ارتباط برقرار کند.
       • جمله از یک ایده به خوبی استفاده کرده اما تنوع ایده های داده شده کم است.
       • جمله از دو ایده متفاوت برای بیان موضوع به خوبی استفاده کرده است.
       • جمله از چندین ایده متفاوت به صورت خلاقانه برای بیان یک موضوع استفاده کرده است.

    2. آیا جمله از زاویه دید جدیدی به موضوع نگاه می‌کند؟
       • جمله از دیدگاهی کاملاً کلیشه‌ای بیان شده است.
       • جمله تا حدی دیدگاه متفاوتی دارد اما همچنان رایج است.
       • جمله دارای زاویه دید متفاوتی است اما می‌توان آن را بهتر کرد.
       • جمله دیدگاه جالب و خلاقانه‌ای ارائه می‌دهد.
       • جمله از زاویه‌های کاملاً جدید و غیرمنتظره به موضوع نگاه می‌کند.

    3. آیا جمله از سبک‌های مختلف (مثلاً کنایی، طنز، فلسفی) استفاده می‌کند؟
       • جمله کاملاً ساده و مستقیم بیان شده است.
       • جمله دارای سبک خاصی است اما در این عبارت مناسب است.
       • جمله از یک سبک خاص بهره برده اما می‌توان آن را تقویت کرد.
       • جمله دارای ترکیبی از سبک‌های ادبی است.
       • جمله به زیبایی از چندین سبک بیانی استفاده کرده است.

    **مهم:** برای هر سوال یک امتیاز عددی بین 1 تا 5 بدهید و فقط عدد بنویسید (مثلاً 1، 2، 3، 4 یا 5). هیچ توضیحی ندهید و از اعشاری (مانند 4.5 یا 1.33) استفاده نکنید.

    جمله: "{sentence}"  

    پاسخ خود را فقط در فرمت زیر بدهید:  
    ایده های متفاوت: X
    زاویه دید جدید: X  
    استفاده از سبک‌های مختلف: X
    """  

    try:  
        response = client.chat.completions.create(  
            model=MODEL,  
            messages=[  
                {"role": "system", "content": "شما یک کارشناس در ارزیابی خلاقیت متون ادبی هستید."},  
                {"role": "user", "content": prompt}  
            ],  
            max_tokens=50,  
        )  
        
        generated_text = response.choices[0].message.content.strip().lower()  
        print(generated_text)  
        
        scores = {}  
        for line in generated_text.split("\n"):  
            if ":" in line:  
                key, value = line.split(":")  
                key, value = key.strip(), value.strip()  

                if key in ["ایده های متفاوت", "زاویه دید جدید", "استفاده از سبک‌های مختلف"]:  
                    try:  
                        value = int(value)  
                        if 1 <= value <= 5:  
                            scores[key] = value  
                        else:  
                            scores[key] = None 
                    except ValueError:  
                        scores[key] = None   

        if len(scores) != 3:  
            return {crit: None for crit in ["ایده های متفاوت", "زاویه دید جدید", "استفاده از سبک‌های مختلف"]}  

        return scores  

    except Exception as e:  
        print(f"❌ خطا در پردازش جمله: {sentence}\n🔹 خطا: {str(e)}")  
        return {crit: None for crit in ["ایده های متفاوت", "زاویه دید جدید", "استفاده از سبک‌های مختلف"]}  


In [ ]:
scores_list_flexibility = []
flexibility_criteria = ["ایده های متفاوت", "زاویه دید جدید", "استفاده از سبک‌های مختلف"]  

for sentence in tqdm(df["Sentence"], desc="Scoring Sentences with GPT-4o"):
    flexibility_scores = claude_score_flexibility(sentence)  
    flexibility_scores["sentence"] = sentence
    
    if all(v is not None for v in flexibility_scores.values()):  

        flexibility_scores["Flexibility Score"] = sum(flexibility_scores[crit] for crit in flexibility_criteria) / len(flexibility_criteria)
    else:
        flexibility_scores["Flexibility Score"] = None  

    scores_list_flexibility.append(flexibility_scores)
    time.sleep(1)  

flexibility_df = pd.DataFrame(scores_list_flexibility)


In [ ]:
merge_flexibility = pd.merge(merge_fluency, flexibility_df, on='sentence',how='left')

# Score Elaboration

In [ ]:
def claude_score_elaboration(sentence):  
    prompt = f"""  
    شما باید یک جمله را بر اساس ۳ زیرمعیار مرتبط با "Elaboration" ارزیابی کنید.  
    لطفاً برای هر معیار یک عدد بین **۱ تا ۵** بدهید. **فقط عدد را بدهید و هیچ توضیحی اضافه نکنید.**
    کمترین نمره عدد ۱ و بیشترین نمره عدد ۵ است** 

    1. آیا جمله به جزییات پرداخته و از واژگان متنوع استفاده کرده است؟  
       • جمله به جزئیات نمی‌پردازد و از واژگان بسیار محدود و تکراری بهره می‌برد  
       • جمله بدون اشاره به جزئیات کافی، از واژگان نسبتا یکنواختی استفاده کرده است  
       • جمله به برخی جزئیات پرداخته است، اما استفاده از واژگان محدود است و تنوع خاصی ندارد  
       • جمله جزئیات مشخصی ارائه می‌دهد، اما واژگان آن نیاز به تنوع بیشتری دارد  
       • جمله به‌خوبی به جزئیات پرداخته و واژگان متنوع و جذابی دارد  

    2. آیا جمله باعث ایجاد یک تصویر ذهنی واضح در خواننده می‌شود؟  
       • جمله هیچ تصویری در ذهن خواننده ایجاد نمی‌کند  
       • جمله تا حدی تصویر ایجاد می‌کند اما خیلی واضح نیست  
       • جمله تصویری متوسط ایجاد می‌کند اما جای تقویت دارد  
       • جمله به‌خوبی یک تصویر ذهنی را منتقل می‌کند  
       • جمله کاملاً تصویری، زنده و واضح است  

    3. آیا جمله حس خاصی (مثلاً عشق، غم، امید) را به خوبی منتقل می‌کند؟  
       • جمله هیچ حس خاصی ندارد و خشک و بی‌روح است  
       • جمله احساس خفیفی دارد اما ضعیف است  
       • جمله احساس مشخصی را منتقل می‌کند اما می‌توان آن را تقویت کرد  
       • جمله احساس قوی و ملموسی ایجاد می‌کند  
       • جمله به‌شدت احساسی و تأثیرگذار است  

    **مهم:** برای هر سوال یک امتیاز عددی بین 1 تا 5 بدهید و فقط عدد بنویسید (مثلاً 1، 2، 3، 4 یا 5). هیچ توضیحی ندهید و از اعشاری (مانند 4.5 یا 1.33) استفاده نکنید.  

    جمله: "{sentence}"  

   پاسخ خود را در فرمت زیر بدهید:  
    پرداخت به جزییات: X  
    ایجاد تصویر ذهنی: X  
    انتقال حس خاص: X
    """  

    try:  
        response = client.chat.completions.create(  
            model=MODEL,  
            messages=[  
                {"role": "system", "content": "شما یک کارشناس در ارزیابی خلاقیت متون ادبی هستید."},  
                {"role": "user", "content": prompt}  
            ],  
            max_tokens=50,  
        )  
        
        generated_text = response.choices[0].message.content.strip().lower()  
        print(generated_text)  
        
        # استخراج نمرات از پاسخ  
        scores = {}  
        for line in generated_text.split("\n"):  
            if ":" in line:  
                key, value = line.split(":")  
                key, value = key.strip(), value.strip()  

                if key in ["پرداخت به جزییات", "ایجاد تصویر ذهنی", "انتقال حس خاص"]:  
                    try:  
                        value = int(value) 
                        if 1 <= value <= 5:  
                            scores[key] = value  
                        else:  
                            scores[key] = None   
                    except ValueError:  
                        scores[key] = None  
                        
        if len(scores) != 3:  
            return {crit: None for crit in ["پرداخت به جزییات", "ایجاد تصویر ذهنی", "انتقال حس خاص"]}  

        return scores  

    except Exception as e:  
        print(f"❌ خطا در پردازش جمله: {sentence}\n🔹 خطا: {str(e)}")  
        return {crit: None for crit in ["پرداخت به جزییات", "ایجاد تصویر ذهنی", "انتقال حس خاص"]}  


In [ ]:
scores_list_elaboration = []
elaboration_criteria = ["پرداخت به جزییات", "ایجاد تصویر ذهنی", "انتقال حس خاص"] 

for sentence in tqdm(df["Sentence"], desc="Scoring Sentences with Cluade 3.5"):
    elaboration_scores = claude_score_elaboration(sentence)  
    elaboration_scores["sentence"] = sentence
    
    if all(v is not None for v in elaboration_scores.values()):  

        elaboration_scores["Elaboration Score"] = sum(elaboration_scores[crit] for crit in elaboration_criteria) / len(elaboration_criteria)
    else:
        elaboration_scores["Elaboration Score"] = None 

    scores_list_elaboration.append(elaboration_scores)
    time.sleep(1)  

elaboration_df = pd.DataFrame(scores_list_elaboration)


# Merge Dataset

In [ ]:
merge_dataset = pd.merge(merge_flexibility, elaboration_df, on='sentence',how='left')

# Save as CSV

In [ ]:
# Save in CSV
merge_dataset.to_csv("claude_scores_{model_name}.csv", index=False)

print("✅ امتیازدهی کامل شد! فایل خروجی claude_scores_{model_name}.csv ذخیره شد.")